In [1]:
from __future__ import division
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import re
import pickle
import matplotlib.pyplot as plt
from xgboost import plot_importance
import plotly.graph_objects as go
from tqdm.notebook import tqdm
from sklearn.model_selection import RepeatedKFold
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix, mean_squared_error
import seaborn as sns
from statannot import add_stat_annotation
from sklearn.linear_model import Lasso, MultiTaskLasso, Ridge, ElasticNet

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression, f_regression

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.decomposition import PCA

from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

from sklearn import metrics


from skopt import BayesSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import statsmodels.api as sm

import time

import scipy

from sklearn.svm import SVR

from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestRegressor

from statsmodels.stats.multitest import fdrcorrection

from statsmodels.stats import multitest

from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.ensemble import ExtraTreesClassifier

from collections import Counter
import heapq

import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor

import re

from sklearn.preprocessing import MinMaxScaler

import os

In [2]:
conn_df =  pd.read_excel('Data/connectivity.xlsx', index_col=0)
conn_df

,transitivity_mimat,transitivity_cmimat,transitivity_omat,global_efficiency_mimat,global_efficiency_cmimat,global_efficiency_omat,density_mimat,density_cmimat,density_omat,small_worldness_mimat,small_worldness_cmimat,small_worldness_omat,Age,Sex,Educ,MMSE,GINI,ODQ
0,0.583842,0.104833,0.001365,0.574837,0.188519,-3.741949e-08,1.02439,1.02439,1.02439,0.750617,0.074869,0.000074,33.0,1.0,19.0,30.0,40.70,99.282421
1,0.664119,0.394995,0.001960,0.655410,0.468515,4.039244e-04,1.02439,1.02439,1.02439,0.811830,0.368709,0.000119,57.0,2.0,13.0,NaN,40.70,99.739738
2,0.557863,0.132563,0.001388,0.549592,0.206538,-6.169134e-08,1.02439,1.02439,1.02439,0.727810,0.121368,0.000075,32.0,1.0,10.0,NaN,40.70,99.361739
3,0.663217,0.315265,0.001743,0.653910,0.373556,1.801059e-04,1.02439,1.02439,1.02439,0.811901,0.290259,0.000114,21.0,1.0,13.0,NaN,40.70,99.561273
4,0.657107,0.249122,0.001494,0.648036,0.307371,3.757728e-05,1.02439,1.02439,1.02439,0.806713,0.222110,0.000079,19.0,1.0,13.0,30.0,40.70,99.822774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,0.446310,0.040522,0.001126,0.439565,0.138220,-2.311560e-08,1.02439,1.02439,1.02439,0.624086,0.035083,0.000059,57.0,2.0,17.0,30.0,44.92,98.650235
1429,0.497250,0.095217,0.001228,0.489429,0.182804,2.682689e-05,1.02439,1.02439,1.02439,0.675007,0.092575,0.000065,48.0,2.0,17.0,28.0,44.92,99.599221
1430,0.475298,0.049578,0.001177,0.467717,0.144292,-1.349355e-08,1.02439,1.02439,1.02439,0.654477,0.040671,0.000060,57.0,2.0,17.0,29.0,44.92,99.675799
1431,0.433723,0.029930,0.001184,0.427825,0.129571,4.167022e-05,1.02439,1.02439,1.02439,0.606633,0.026680,0.000061,69.0,2.0,12.0,30.0,44.92,99.641446


## Statistical significance check

In [3]:
outcomes_list = list(conn_df.iloc[:,0:-9])
df_metrics_slopes = pd.DataFrame(columns=outcomes_list)
df_metrics_pval = pd.DataFrame(columns=outcomes_list)
df_metrics_dir = pd.DataFrame(columns=outcomes_list)
df_metrics_slopes

,transitivity_mimat,transitivity_cmimat,transitivity_omat,global_efficiency_mimat,global_efficiency_cmimat,global_efficiency_omat,density_mimat,density_cmimat,density_omat


In [4]:
from scipy.stats import linregress

df_metrics_slopes = pd.DataFrame(columns=outcomes_list)

for variable_y in outcomes_list:
    
    c_conn_df_mmse = conn_df[['GINI', variable_y]]
    
    c_conn_df_mmse = c_conn_df_mmse.dropna()
    c_conn_df_mmse.reset_index(drop=True)
    
    y_conn_1A = c_conn_df_mmse[variable_y]
    X_conn_1A = c_conn_df_mmse[['GINI']]
    
    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_conn_1A)
    X_conn_1A = pd.DataFrame(scaling_data, columns= X_conn_1A.columns, index = X_conn_1A.index)

    X_conn_1A =  X_conn_1A.copy()

    slope, intercept, r_value, p_value, std_err = linregress(X_conn_1A['GINI'], y_conn_1A)
    
    p_value*=210
    if(p_value<=0.05):
        df_metrics_slopes.loc['GINI', variable_y] =  str(np.round(slope,3)) + ' (' + str(np.round(p_value,3))+')' #slope
    
    df_metrics_pval.loc['GINI', variable_y] =  p_value
    df_metrics_dir.loc['GINI', variable_y] =  slope
    
for variable_y in outcomes_list:
    for variable_x in ['Age', 'Educ', 'Sex', 'MMSE']:
    
        c_conn_df_mmse = conn_df[['GINI', variable_x, variable_y]] # Esto lo hago para que sean los mismo datos que en la regresión

        c_conn_df_mmse = c_conn_df_mmse.dropna()
        c_conn_df_mmse.reset_index(drop=True)
        
        y_conn_1A = c_conn_df_mmse[variable_y]
        X_conn_1A = c_conn_df_mmse[['GINI', variable_x]]
        
        scaler = MinMaxScaler([0.05, 0.95])
        scaling_data = scaler.fit_transform(X_conn_1A)
        X_conn_1A = pd.DataFrame(scaling_data, columns= X_conn_1A.columns, index = X_conn_1A.index)

        X_conn_1A =  X_conn_1A.copy()

        slope, intercept, r_value, p_value, std_err = linregress(X_conn_1A[variable_x],y_conn_1A)
    
        p_value*=210
        if(p_value<=0.05):
            df_metrics_slopes.loc[variable_x, variable_y] =  str(np.round(slope,3)) + ' (' + str(np.round(p_value,3))+')' #slope
     
        df_metrics_pval.loc[variable_x, variable_y] =  p_value
        df_metrics_dir.loc[variable_x, variable_y] =  slope

In [5]:
df_metrics_slopes = df_metrics_slopes.T
df_metrics_pval = df_metrics_pval.T
df_metrics_dir = df_metrics_dir.T
df_metrics_dir

,GINI,Age,Educ,Sex,MMSE
transitivity_mimat,0.007025,-0.136356,-0.110063,-0.022081,0.055234
transitivity_cmimat,0.001101,-0.091881,-0.068669,-0.020166,0.050544
transitivity_omat,0.000664,0.002487,0.002195,0.000456,-0.000935
global_efficiency_mimat,0.008068,-0.127716,-0.101644,-0.020533,0.052666
global_efficiency_cmimat,0.046546,-0.127632,-0.104343,-0.016811,0.035658
global_efficiency_omat,0.000133,0.000991,0.000964,0.0002,-0.000277
density_mimat,-0.0,-0.0,-0.0,-0.0,-0.0
density_cmimat,-0.0,-0.0,-0.0,-0.0,-0.0
density_omat,-0.0,-0.0,-0.0,-0.0,-0.0


In [6]:
df_metrics_pval

,GINI,Age,Educ,Sex,MMSE
transitivity_mimat,113.824758,0.0,0.262454,0.038057,1.193062
transitivity_cmimat,188.783687,0.0,1.532548,0.00112,0.136292
transitivity_omat,26.37683,0.0,18.7004,8.602928,42.529795
global_efficiency_mimat,93.806499,0.0,0.25095,0.032338,0.758004
global_efficiency_cmimat,0.002705,0.0,0.197813,0.438429,10.269686
global_efficiency_omat,103.242671,0.000001,19.505862,9.217895,70.058758
density_mimat,210.0,210.0,210.0,210.0,210.0
density_cmimat,210.0,210.0,210.0,210.0,210.0
density_omat,210.0,210.0,210.0,210.0,210.0


In [7]:
df_metrics_GINI = df_metrics_slopes[['GINI']]
df_metrics_GINI = df_metrics_GINI.copy()
df_metrics_GINI.dropna(inplace=True)

In [8]:
outcomes_list = list(df_metrics_GINI.index)

In [9]:
def classifier_ROC(X_, y, fig_show = True, params_ = None):
    
    X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2, random_state=10000, stratify=y)

    #lista = list(sfs_.k_feature_names_)

    #X_train = X_train[lista]
    #X_test = X_test[lista]

    cv    = RepeatedKFold(n_splits=10, n_repeats=2, random_state=101)
    folds = [(train,test) for train, test in cv.split(X_train, y_train)]
    metrics = ['auc', 'fpr', 'tpr', 'thresholds', 'f1_score', 'accuracy_score', 'recall_score', 'precision_score', 'confusion_matrix', 'model']
    results = {
        'train': {m:[] for m in metrics},
        'val'  : {m:[] for m in metrics},
        'test' : {m:[] for m in metrics}
    }
    X_train = pd.DataFrame(X_train)
    y_train = pd.DataFrame(y_train)


    if(params_!=None):
        params = params_
    else:
        params = {
            'objective'   : 'binary:logistic',
            'eval_metric' : 'logloss'
        }


    plt.rcParams["figure.figsize"] = (10,10)
    dtest = xgb.DMatrix(X_test, label=y_test)
    for train, test in tqdm(folds, total=len(folds)):
        dtrain = xgb.DMatrix(X_train.iloc[train,:], label=y_train.iloc[train])
        dval   = xgb.DMatrix(X_train.iloc[test,:], label=y_train.iloc[test])
        model  = xgb.train(
            dtrain                = dtrain,
            params                = params, 
            evals                 = [(dtrain, 'train'), (dval, 'val')],
            num_boost_round       = 1000,
            verbose_eval          = False,
            early_stopping_rounds = 10,
        )
        sets = [dtrain, dval, dtest]

        for i,ds in enumerate(results.keys()):
            y_preds              = model.predict(sets[i])
            labels               = sets[i].get_label()
            fpr, tpr, thresholds = roc_curve(labels, y_preds)
            results[ds]['fpr'].append(fpr)
            results[ds]['tpr'].append(tpr)
            results[ds]['thresholds'].append(thresholds)
            results[ds]['auc'].append(roc_auc_score(labels, y_preds))

            results[ds]['f1_score'].append(f1_score(labels, np.round(y_preds)))
            results[ds]['accuracy_score'].append(accuracy_score(labels, np.round(y_preds)))
            results[ds]['recall_score'].append(recall_score(labels, np.round(y_preds)))
            results[ds]['precision_score'].append(precision_score(labels, np.round(y_preds)))
            results[ds]['confusion_matrix'].append(confusion_matrix(labels, np.round(y_preds)))
            results[ds]['model'].append(model)


    kind = 'test'
    c_fill      = 'rgba(52, 152, 219, 0.2)'
    c_line      = 'rgba(52, 152, 219, 0.5)'
    c_line_main = 'rgba(41, 128, 185, 1.0)'
    c_grid      = 'rgba(189, 195, 199, 0.5)'
    c_annot     = 'rgba(149, 165, 166, 0.5)'
    c_highlight = 'rgba(192, 57, 43, 1.0)'

    fpr_mean    = np.linspace(0, 1, 100)
    interp_tprs = []

    for i in range(10*2):
        fpr           = results[kind]['fpr'][i]
        tpr           = results[kind]['tpr'][i]
        interp_tpr    = np.interp(fpr_mean, fpr, tpr)
        interp_tpr[0] = 0.0
        interp_tprs.append(interp_tpr)

    tpr_mean     = np.mean(interp_tprs, axis=0)
    tpr_mean[-1] = 1.0
    tpr_std      = 2*np.std(interp_tprs, axis=0)
    tpr_upper    = np.clip(tpr_mean+tpr_std, 0, 1)
    tpr_lower    = tpr_mean-tpr_std
    auc          = np.mean(results[kind]['auc'])
    
    roc_dict = {}
    roc_dict['tpr_mean'] = tpr_mean
    roc_dict['tpr_std'] = tpr_std
    roc_dict['tpr_upper'] = tpr_upper
    roc_dict['tpr_lower'] = tpr_lower
    roc_dict['auc'] = auc

    if(fig_show):
        fig = go.Figure([
            go.Scatter(
                x          = fpr_mean,
                y          = tpr_upper,
                line       = dict(color=c_line, width=1),
                hoverinfo  = "skip",
                showlegend = False,
                name       = 'upper'),
            go.Scatter(
                x          = fpr_mean,
                y          = tpr_lower,
                fill       = 'tonexty',
                fillcolor  = c_fill,
                line       = dict(color=c_line, width=1),
                hoverinfo  = "skip",
                showlegend = False,
                name       = 'lower'),
            go.Scatter(
                x          = fpr_mean,
                y          = tpr_mean,
                line       = dict(color=c_line_main, width=2),
                hoverinfo  = "skip",
                showlegend = True,
                name       = f'AUC: {auc:.3f}')
        ])

        fig.add_shape(
            type ='line', 
            line =dict(dash='dash'),
            x0=0, x1=1, y0=0, y1=1
        )

        fig.update_layout(
            template    = 'plotly_white', 
            title_x     = 0.5,
            xaxis_title = "1 - Specificity",
            yaxis_title = "Sensitivity",
            width       = 800,
            height      = 800,
            legend      = dict(
                yanchor="bottom", 
                xanchor="right", 
                x=0.95,
                y=0.01,
            )
        )

        fig.update_yaxes(
            range       = [0, 1],
            gridcolor   = c_grid,
            scaleanchor = "x", 
            scaleratio  = 1,
            linecolor   = 'black')

        fig.update_xaxes(
            range       = [0, 1],
            gridcolor   = c_grid,
            constrain   = 'domain',
            linecolor   = 'black')



        fig.show()
        
    else:
        print(y.name, 'AUC: ', np.round(auc,2), '+-( ', np.round(np.std(results[kind]['auc']),2), ')', X_.shape)

    return (model, results, np.round(auc,2), roc_dict)

In [10]:
from skopt import BayesSearchCV
def SearchBestModel(case_x, case_y):

    X_train, X_test, y_train, y_test = train_test_split(case_x, case_y, test_size=.2, random_state=1)

    opt_XGB = BayesSearchCV(
            XGBClassifier(),
            {
                'objective'   : ['binary:logistic'],
                'eval_metric' : ['logloss', 'error', 'auc', 'aucpr'],
                'learning_rate': (0.01, 0.1, 0.2),
                'max_depth': (3, 4, 5),
                'subsample': (0.8, 0.9, 1.0),
                'colsample_bytree': (0.8, 0.9, 1.0),
                'reg_alpha': (0, 0.1, 1.0),
                'reg_lambda': (0, 0.1, 1.0),
                #'gamma': (0.001, 0.01, 0.1, 1, 10),
            },
                          
            #n_iter=10,
            #cv=3,
            random_state=1
        )

    opt_XGB.fit(X_train, y_train)


    print('XGB')
    print("val. score: %s" % opt_XGB.best_score_)
    print("test score: %s" % opt_XGB.score(X_test, y_test))
    print("best parameters: %s" % str(opt_XGB.best_params_))
    print('---------------------------------------------\n')
    
    return opt_XGB

### Classification

In [11]:

gini_results_preliminar = {}
gini_results_auc_preliminar = {}
for outcome in outcomes_list:

    conn_df_mmse = conn_df[[outcome, 'GINI']]
    conn_df_mmse = conn_df_mmse.dropna()
    conn_df_mmse.reset_index(drop=True)

   
    mediana = conn_df_mmse[outcome].median()
    conn_df_mmse[outcome + '_b'] = (conn_df_mmse[outcome] > mediana).astype(int)
    
    y_conn_1A = conn_df_mmse[outcome + '_b']
    X_conn_1A = conn_df_mmse[['GINI']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_conn_1A)
    X_conn_1A_s = pd.DataFrame(scaling_data, columns= X_conn_1A.columns, index = X_conn_1A.index)
    
    X_conn_1A =  X_conn_1A_s.copy()
    
    try:
        r = classifier_ROC(X_conn_1A, y_conn_1A, False)
        gini_results_preliminar[outcome] = r
        gini_results_auc_preliminar[outcome] = r[2]
    except:
        print('Bad: ',outcome + '_b')

  0%|          | 0/20 [00:00<?, ?it/s]

global_efficiency_cmimat_b AUC:  0.79 +-(  0.0 ) (1361, 1)


In [12]:
r = classifier_ROC(X_conn_1A, y_conn_1A, False)

  0%|          | 0/20 [00:00<?, ?it/s]

global_efficiency_cmimat_b AUC:  0.79 +-(  0.0 ) (1361, 1)


In [13]:
top_10_keys_GINI_preliminar = heapq.nlargest(10, gini_results_auc_preliminar, key=gini_results_auc_preliminar.get)
top_10_keys_GINI_preliminar

['global_efficiency_cmimat']

In [14]:
gini_results = {}
gini_results_auc = {}
gini_paragram = {}
for outcome in top_10_keys_GINI_preliminar:

    conn_df_mmse = conn_df[[outcome, 'GINI']]
    conn_df_mmse = conn_df_mmse.dropna()
    conn_df_mmse.reset_index(drop=True)

   
    mediana = conn_df_mmse[outcome].median()
    conn_df_mmse[outcome + '_b'] = (conn_df_mmse[outcome] > mediana).astype(int)
    
    y_conn_1A = conn_df_mmse[outcome + '_b']
    X_conn_1A = conn_df_mmse[['GINI']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_conn_1A)
    X_conn_1A_s = pd.DataFrame(scaling_data, columns= X_conn_1A.columns, index = X_conn_1A.index)
    
    X_conn_1A =  X_conn_1A_s.copy()

    
    opt_XGB = SearchBestModel(X_conn_1A, y_conn_1A)
    params_ = dict(opt_XGB.best_params_)


    r = classifier_ROC(X_conn_1A, y_conn_1A, False, params_)
    gini_results[outcome] = r
    gini_results_auc[outcome] = r[2]
    gini_paragram[outcome] = params_


XGB
val. score: 0.7453980467593962
test score: 0.7728937728937729
best parameters: OrderedDict([('colsample_bytree', 0.9), ('eval_metric', 'auc'), ('learning_rate', 0.1), ('max_depth', 3), ('objective', 'binary:logistic'), ('reg_alpha', 1.0), ('reg_lambda', 1.0), ('subsample', 1.0)])
---------------------------------------------



  0%|          | 0/20 [00:00<?, ?it/s]

global_efficiency_cmimat_b AUC:  0.78 +-(  0.01 ) (1361, 1)


In [15]:
params_

{'colsample_bytree': 0.9,
 'eval_metric': 'auc',
 'learning_rate': 0.1,
 'max_depth': 3,
 'objective': 'binary:logistic',
 'reg_alpha': 1.0,
 'reg_lambda': 1.0,
 'subsample': 1.0}

In [16]:
y_conn_1A.value_counts()

0    681
1    680
Name: global_efficiency_cmimat_b, dtype: int64

In [17]:
top_10_keys_GINI = heapq.nlargest(10, gini_results_auc, key=gini_results_auc.get)
top_10_keys_GINI

['global_efficiency_cmimat']

## Classification:  Gini + Age

In [18]:

gini_age_results_preliminar = {}
gini_age_results_auc_preliminar = {}
for outcome in outcomes_list:

    conn_df_mmse = conn_df[[outcome, 'GINI', 'Age']]
    conn_df_mmse = conn_df_mmse.dropna()
    conn_df_mmse.reset_index(drop=True)

   
    mediana = conn_df_mmse[outcome].median()
    conn_df_mmse[outcome + '_b'] = (conn_df_mmse[outcome] > mediana).astype(int)
    
    y_conn_1A = conn_df_mmse[outcome + '_b']
    X_conn_1A = conn_df_mmse[['GINI', 'Age']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_conn_1A)
    X_conn_1A_s = pd.DataFrame(scaling_data, columns= X_conn_1A.columns, index = X_conn_1A.index)
    
    X_conn_1A =  X_conn_1A_s.copy()
    
    try:
        r = classifier_ROC(X_conn_1A, y_conn_1A, False)
        gini_age_results_preliminar[outcome] = r
        gini_age_results_auc_preliminar[outcome] = r[2]
    except:
        print('Bad: ',outcome + '_b')

  0%|          | 0/20 [00:00<?, ?it/s]

global_efficiency_cmimat_b AUC:  0.78 +-(  0.0 ) (1361, 2)


In [19]:
top_10_keys_GINI_AGE_preliminar = heapq.nlargest(10, gini_age_results_auc_preliminar, key=gini_age_results_auc_preliminar.get)

In [20]:
gini_age_results = {}
gini_age_results_auc = {}
gini_age_paragram = {}
for outcome in top_10_keys_GINI_AGE_preliminar:

    conn_df_mmse = conn_df[[outcome, 'GINI', 'Age']]
    conn_df_mmse = conn_df_mmse.dropna()
    conn_df_mmse.reset_index(drop=True)

   
    mediana = conn_df_mmse[outcome].median()
    conn_df_mmse[outcome + '_b'] = (conn_df_mmse[outcome] > mediana).astype(int)
    
    y_conn_1A = conn_df_mmse[outcome + '_b']
    X_conn_1A = conn_df_mmse[['GINI', 'Age']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_conn_1A)
    X_conn_1A_s = pd.DataFrame(scaling_data, columns= X_conn_1A.columns, index = X_conn_1A.index)
    
    X_conn_1A =  X_conn_1A_s.copy()

    
    opt_XGB = SearchBestModel(X_conn_1A, y_conn_1A)
    params_ = dict(opt_XGB.best_params_)


    r = classifier_ROC(X_conn_1A, y_conn_1A, False, params_)
    gini_age_results[outcome] = r
    gini_age_results_auc[outcome] = r[2]
    gini_age_paragram[outcome] = params_


XGB
val. score: 0.7518496596626221
test score: 0.7692307692307693
best parameters: OrderedDict([('colsample_bytree', 1.0), ('eval_metric', 'logloss'), ('learning_rate', 0.2), ('max_depth', 3), ('objective', 'binary:logistic'), ('reg_alpha', 1.0), ('reg_lambda', 0.0), ('subsample', 1.0)])
---------------------------------------------



  0%|          | 0/20 [00:00<?, ?it/s]

global_efficiency_cmimat_b AUC:  0.78 +-(  0.0 ) (1361, 2)


In [21]:
top_10_keys_GINI_age = heapq.nlargest(10, gini_age_results_auc, key=gini_age_results_auc.get)
top_10_keys_GINI_age

['global_efficiency_cmimat']

## Classification:  Gini + Educ

In [22]:

gini_educ_results_preliminar = {}
gini_educ_results_auc_preliminar = {}
for outcome in outcomes_list:

    conn_df_mmse = conn_df[[outcome, 'GINI', 'Educ']]
    conn_df_mmse = conn_df_mmse.dropna()
    conn_df_mmse.reset_index(drop=True)

   
    mediana = conn_df_mmse[outcome].median()
    conn_df_mmse[outcome + '_b'] = (conn_df_mmse[outcome] > mediana).astype(int)
    
    y_conn_1A = conn_df_mmse[outcome + '_b']
    X_conn_1A = conn_df_mmse[['GINI', 'Educ']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_conn_1A)
    X_conn_1A_s = pd.DataFrame(scaling_data, columns= X_conn_1A.columns, index = X_conn_1A.index)
    
    X_conn_1A =  X_conn_1A_s.copy()
    
    try:
        r = classifier_ROC(X_conn_1A, y_conn_1A, False)
        gini_educ_results_preliminar[outcome] = r
        gini_educ_results_auc_preliminar[outcome] = r[2]
    except:
        print('Bad: ',outcome + '_b')

  0%|          | 0/20 [00:00<?, ?it/s]

global_efficiency_cmimat_b AUC:  0.8 +-(  0.0 ) (1303, 2)


In [23]:
top_10_keys_GINI_educ_preliminar = heapq.nlargest(10, gini_educ_results_auc_preliminar, key=gini_educ_results_auc_preliminar.get)

In [24]:
gini_educ_results = {}
gini_educ_results_auc = {}
gini_educ_paragram = {}
for outcome in top_10_keys_GINI_educ_preliminar:

    conn_df_mmse = conn_df[[outcome, 'GINI', 'Educ']]
    conn_df_mmse = conn_df_mmse.dropna()
    conn_df_mmse.reset_index(drop=True)

   
    mediana = conn_df_mmse[outcome].median()
    conn_df_mmse[outcome + '_b'] = (conn_df_mmse[outcome] > mediana).astype(int)
    
    y_conn_1A = conn_df_mmse[outcome + '_b']
    X_conn_1A = conn_df_mmse[['GINI', 'Educ']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_conn_1A)
    X_conn_1A_s = pd.DataFrame(scaling_data, columns= X_conn_1A.columns, index = X_conn_1A.index)
    
    X_conn_1A =  X_conn_1A_s.copy()

    
    opt_XGB = SearchBestModel(X_conn_1A, y_conn_1A)
    params_ = dict(opt_XGB.best_params_)


    r = classifier_ROC(X_conn_1A, y_conn_1A, False, params_)
    gini_educ_results[outcome] = r
    gini_educ_results_auc[outcome] = r[2]
    gini_educ_paragram[outcome] = params_


XGB
val. score: 0.7582121825542878
test score: 0.7471264367816092
best parameters: OrderedDict([('colsample_bytree', 0.8), ('eval_metric', 'error'), ('learning_rate', 0.1), ('max_depth', 5), ('objective', 'binary:logistic'), ('reg_alpha', 0.1), ('reg_lambda', 1.0), ('subsample', 0.9)])
---------------------------------------------



  0%|          | 0/20 [00:00<?, ?it/s]

global_efficiency_cmimat_b AUC:  0.81 +-(  0.0 ) (1303, 2)


In [25]:
top_10_keys_gini_educ = heapq.nlargest(10, gini_educ_results_auc, key=gini_educ_results_auc.get)
top_10_keys_gini_educ

['global_efficiency_cmimat']

## Classification:  Gini + Sex

In [26]:

gini_sex_results_preliminar = {}
gini_sex_results_auc_preliminar = {}
for outcome in outcomes_list:

    conn_df_mmse = conn_df[[outcome, 'GINI', 'Sex']]
    conn_df_mmse = conn_df_mmse.dropna()
    conn_df_mmse.reset_index(drop=True)

   
    mediana = conn_df_mmse[outcome].median()
    conn_df_mmse[outcome + '_b'] = (conn_df_mmse[outcome] > mediana).astype(int)
    
    y_conn_1A = conn_df_mmse[outcome + '_b']
    X_conn_1A = conn_df_mmse[['GINI', 'Sex']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_conn_1A)
    X_conn_1A_s = pd.DataFrame(scaling_data, columns= X_conn_1A.columns, index = X_conn_1A.index)
    
    X_conn_1A =  X_conn_1A_s.copy()
    
    try:
        r = classifier_ROC(X_conn_1A, y_conn_1A, False)
        gini_sex_results_preliminar[outcome] = r
        gini_sex_results_auc_preliminar[outcome] = r[2]
    except:
        print('Bad: ',outcome + '_b')

  0%|          | 0/20 [00:00<?, ?it/s]

global_efficiency_cmimat_b AUC:  0.82 +-(  0.0 ) (1321, 2)


In [27]:
top_10_keys_GINI_sex_preliminar = heapq.nlargest(10, gini_sex_results_auc_preliminar, key=gini_sex_results_auc_preliminar.get)

In [28]:
gini_sex_results = {}
gini_sex_results_auc = {}
gini_sex_paragram = {}
for outcome in top_10_keys_GINI_sex_preliminar:

    conn_df_mmse = conn_df[[outcome, 'GINI', 'Sex']]
    conn_df_mmse = conn_df_mmse.dropna()
    conn_df_mmse.reset_index(drop=True)

   
    mediana = conn_df_mmse[outcome].median()
    conn_df_mmse[outcome + '_b'] = (conn_df_mmse[outcome] > mediana).astype(int)
    
    y_conn_1A = conn_df_mmse[outcome + '_b']
    X_conn_1A = conn_df_mmse[['GINI', 'Sex']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_conn_1A)
    X_conn_1A_s = pd.DataFrame(scaling_data, columns= X_conn_1A.columns, index = X_conn_1A.index)
    
    X_conn_1A =  X_conn_1A_s.copy()

    
    opt_XGB = SearchBestModel(X_conn_1A, y_conn_1A)
    params_ = dict(opt_XGB.best_params_)
    

    r = classifier_ROC(X_conn_1A, y_conn_1A, False, params_)
    gini_sex_results[outcome] = r
    gini_sex_results_auc[outcome] = r[2]
    gini_sex_paragram[outcome] = params_


XGB
val. score: 0.75
test score: 0.7886792452830189
best parameters: OrderedDict([('colsample_bytree', 0.9), ('eval_metric', 'logloss'), ('learning_rate', 0.2), ('max_depth', 3), ('objective', 'binary:logistic'), ('reg_alpha', 0.1), ('reg_lambda', 0.0), ('subsample', 0.9)])
---------------------------------------------



  0%|          | 0/20 [00:00<?, ?it/s]

global_efficiency_cmimat_b AUC:  0.82 +-(  0.0 ) (1321, 2)


In [29]:
top_10_keys_gini_sex = heapq.nlargest(10, gini_sex_results_auc, key=gini_sex_results_auc.get)
top_10_keys_gini_sex

['global_efficiency_cmimat']

## Classification:  Gini + Cognition

In [30]:

gini_mmse_results_preliminar = {}
gini_mmse_results_auc_preliminar = {}
for outcome in outcomes_list:

    conn_df_mmse = conn_df[[outcome, 'GINI', 'MMSE']]
    conn_df_mmse = conn_df_mmse[conn_df_mmse['MMSE'] >= 21]
    conn_df_mmse = conn_df_mmse.dropna()
    conn_df_mmse.reset_index(drop=True)

   
    mediana = conn_df_mmse[outcome].median()
    conn_df_mmse[outcome + '_b'] = (conn_df_mmse[outcome] > mediana).astype(int)
    
    y_conn_1A = conn_df_mmse[outcome + '_b']
    X_conn_1A = conn_df_mmse[['GINI', 'MMSE']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_conn_1A)
    X_conn_1A_s = pd.DataFrame(scaling_data, columns= X_conn_1A.columns, index = X_conn_1A.index)
    
    X_conn_1A =  X_conn_1A_s.copy()
    
    try:
        r = classifier_ROC(X_conn_1A, y_conn_1A, False)
        gini_mmse_results_preliminar[outcome] = r
        gini_mmse_results_auc_preliminar[outcome] = r[2]
    except:
        print('Bad: ',outcome + '_b')

  0%|          | 0/20 [00:00<?, ?it/s]

global_efficiency_cmimat_b AUC:  0.89 +-(  0.0 ) (738, 2)


In [31]:
top_10_keys_GINI_mmse_preliminar = heapq.nlargest(10, gini_mmse_results_auc_preliminar, key=gini_mmse_results_auc_preliminar.get)

In [32]:
gini_mmse_results = {}
gini_mmse_results_auc = {}
gini_mmse_paragram = {}
for outcome in top_10_keys_GINI_mmse_preliminar:

    conn_df_mmse = conn_df[[outcome, 'GINI', 'MMSE']]
    conn_df_mmse = conn_df_mmse[conn_df_mmse['MMSE'] >= 21]
    conn_df_mmse = conn_df_mmse.dropna()
    conn_df_mmse.reset_index(drop=True)

   
    mediana = conn_df_mmse[outcome].median()
    conn_df_mmse[outcome + '_b'] = (conn_df_mmse[outcome] > mediana).astype(int)
    
    y_conn_1A = conn_df_mmse[outcome + '_b']
    X_conn_1A = conn_df_mmse[['GINI', 'MMSE']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_conn_1A)
    X_conn_1A_s = pd.DataFrame(scaling_data, columns= X_conn_1A.columns, index = X_conn_1A.index)
    
    X_conn_1A =  X_conn_1A_s.copy()

    
    opt_XGB = SearchBestModel(X_conn_1A, y_conn_1A)
    params_ = dict(opt_XGB.best_params_)
    

    r = classifier_ROC(X_conn_1A, y_conn_1A, False, params_)
    gini_mmse_results[outcome] = r
    gini_mmse_results_auc[outcome] = r[2]
    gini_mmse_paragram[outcome] = params_


XGB
val. score: 0.8169491525423729
test score: 0.7567567567567568
best parameters: OrderedDict([('colsample_bytree', 1.0), ('eval_metric', 'logloss'), ('learning_rate', 0.01), ('max_depth', 4), ('objective', 'binary:logistic'), ('reg_alpha', 1.0), ('reg_lambda', 1.0), ('subsample', 1.0)])
---------------------------------------------



  0%|          | 0/20 [00:00<?, ?it/s]

global_efficiency_cmimat_b AUC:  0.9 +-(  0.0 ) (738, 2)


In [33]:
top_10_keys_gini_mmse = heapq.nlargest(10, gini_mmse_results_auc, key=gini_mmse_results_auc.get)
top_10_keys_gini_mmse

['global_efficiency_cmimat']